#### Imports

In [1]:
import locale
import pandas as pd
import numpy as np
import requests
import string
from bs4 import BeautifulSoup as bs
from datetime import datetime
from random import choices, sample

#### Utilizando beatifulSoup para extrair tabela de calendário

In [2]:
data = []

for year in range(2020, 2023):
    page = (
        requests
        .get(f'https://www.timeanddate.com/holidays/china/{year}?hol=4194617')
    )

    soup = bs(page.text, 'html.parser')

    table = soup.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all(['td', 'th'])
        cols = (
            [
                f'{el.text.strip()} de {year}'
                if len(el.text.strip().split('de')) > 1
                else el.text.strip()
                for el in cols
            ]
        )
        data.append([el for el in cols if el])

calendario_china = (
    pd.DataFrame(
        data, columns=[
            'DATA', 'DIA_DA_SEMANA', 'FERIADO', 'TIPO_FERIADO'
        ]
    )
)


#### Organizando o dataset do calendário


In [3]:
# Mostrando as 5 primeiras linhas do dataframe
calendario_china.head()


,DATA,DIA_DA_SEMANA,FERIADO,TIPO_FERIADO
0,None,None,None,None
1,1 de Jan de 2020,Quarta-feira,New Year's Day,National holiday
2,19 de Jan de 2020,Domingo,Special Working Day,Working day on weekend
3,24 de Jan de 2020,Sexta-feira,Spring Festival Eve,National holiday
4,25 de Jan de 2020,Sábado,Lunar New Year,National holiday


Vamos checar os valores missing neste dataframe


In [4]:

# Observando a quantidade de valores None por coluna
calendario_china.isna().sum()

# Como a data é a coluna mais importante deste dataframe, vamos dropar todas as linhas onde a data seja None.
(
    calendario_china
        .dropna(
            subset = ['DATA']
            , inplace = True
        )
)


# Vamos remover os eventos do tipo Working day on weekend. 
# São dias especiais de trabalho onde os chineses precisam ir mesmo sendo finais de semana. 
# Como este calendário traz todos os feriados e eventos importantes, situações como esta acabam ocorrendo.
# Removendo as linhas onde o tipo é Working day on weekend.
(
    calendario_china
        .drop(
            calendario_china[
                calendario_china['TIPO_FERIADO'] == 'Working day on weekend'
            ].index
            , inplace = True
        )
)

Após removermos todas as linhas com dados datas faltantes e os eventos do tipo Working day on weekend , mudaremos formato de apresentação da coluna calendario_china['DATA']

In [5]:

# Como o formato da data esta especificamente no formato pt_BR, foi necessário mudar o padrão POSIX do python para facilitar a atividade de conversão.

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

# Modificando o formato das datas
calendario_china['DATA'] = (
    pd.to_datetime(
        calendario_china['DATA']
            .apply(
                lambda x: 
                datetime.strptime(x, '%d de %b de %Y').strftime('%d/%m/%Y')
            )
        , dayfirst=True
    )
)

# Transformando todos os feriados que possuem golden week no nome em apenas golden week.
calendario_china['FERIADO'] = (
    np.where(
        calendario_china['FERIADO'].str.contains('Golden Week')

        , 'Golden Week'
        , calendario_china['FERIADO']
    )
)

In [6]:
# Criando uma lista de letras maiúsculas
lista_letras = list(string.ascii_uppercase)

# Criando uma lista de feriados
feriados = list(calendario_china['FERIADO'].unique())

# Fazendo o de-para da lista de feriado(com a ordem modificada randomicamente devido a função sample)
# utilizando uma combinação de 2 letras maiúsculas (escolhidas randomicamente devido a função choice)
for feriado in sample(feriados, len(feriados)):
    calendario_china['FERIADO'] = np.where(
        (calendario_china['FERIADO'] == f'{feriado}')
        , choices(lista_letras)[0] + choices(lista_letras)[0]
        , calendario_china['FERIADO']
    )
    
# Escolhendo as colunas que precisaremos para o futuro da análise
calendario_china = calendario_china[['DATA','FERIADO']]

# Renomeando as colunas para padronizarmos a apresentação dos dados para
calendario_china.columns = ['DATE','DS_HOLIDAY_NAME']

### Exportando o dataframe do calendário devidamente organizado para ser utilizado no próximo notebook

In [7]:
calendario_china.to_csv('./dataset/calendario_china.csv',header=True, sep=";", index=False, encoding="utf-8")